In [1]:
from sklearn.datasets import load_boston  
import random

In [2]:
data = load_boston()
X, y = data['data'], data['target']
X_rm = X[:,5]

In [3]:
def price(X_rm, k, b):
    return X_rm * k + b 

## loss

$$ loss = \frac{1}{n}\sum(y_i - \hat{y_i})^2$$

$$ \frac{\partial{loss}}{\partial{k}} = -\frac{2}{n}\sum(y_i - \hat{y_i})x_i$$

$$ \frac{\partial{loss}}{\partial{b}} = -\frac{2}{n}\sum(y_i - \hat{y_i})$$

In [10]:
def loss(y, y_hat):
    return sum((y_i - y_hat_i)**2 for y_i, y_hat_i in zip(list(y), list(y_hat))) / len(list(y))

In [11]:
def partial_k(x, y, y_hat):
    n = len(y)
    gradient = 0
    
    for x_i, y_i, y_hat_i in zip(list(x), list(y), list(y_hat)):
        gradient += (y_i - y_hat_i) * x_i
    return -2 / n *gradient

def partial_b(x, y, y_hat):
    n = len(y)
    gradient = 0
    
    for y_i, y_hat_i in zip(list(y), list(y_hat)):
        gradient += y_i - y_hat_i
    return -2 / n *gradient

In [12]:
tryingtime = 2000
learningrate = 1e-03 
min_loss = float('inf')

current_k = random.random() * 200 - 100
current_b = random.random() * 200 - 100

for i in range(tryingtime):
    price_by_k_and_b = [price(x, current_k, current_b) for x in X_rm]
    current_loss = loss(y, price_by_k_and_b)
    
    if current_loss < min_loss:
        if i % 50 == 0:
            print('When time is : {}, get best_k : {} best_b : {} and the loss is : {}'.format(i, current_k, current_b, current_loss))
            
    k_gradient = partial_k(X_rm, y, price_by_k_and_b)
    b_gradient = partial_b(X_rm, y, price_by_k_and_b)
    current_k = current_k + (-1 * k_gradient) * learningrate
    current_b = current_b + (-1 * k_gradient) * learningrate


When time is : 0, get best_k : 81.79242211085648 best_b : -21.720752542697582 and the loss is : 223341.98648383963
When time is : 50, get best_k : 17.718320557952065 best_b : -85.794854095602 and the loss is : 89.33149018267527
When time is : 100, get best_k : 17.219588143269466 best_b : -86.2935865102846 and the loss is : 76.4350980783701
When time is : 150, get best_k : 17.2157061688769 best_b : -86.29746848467717 and the loss is : 76.43921677624647
When time is : 200, get best_k : 17.215675952823684 best_b : -86.29749870073043 and the loss is : 76.43925516609244
When time is : 250, get best_k : 17.21567571763155 best_b : -86.29749893592263 and the loss is : 76.43925546529054
When time is : 300, get best_k : 17.21567571580089 best_b : -86.2974989377533 and the loss is : 76.43925546761942
When time is : 350, get best_k : 17.215675715786638 best_b : -86.29749893776753 and the loss is : 76.43925546763744
When time is : 400, get best_k : 17.21567571578654 best_b : -86.29749893776759 and 

## Dynamic Programming

In [1]:
from collections import defaultdict
from functools import wraps

In [2]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]
price = defaultdict(int)
for i, p in enumerate(original_price):
    price[i + 1] = p

In [3]:
def r(n):
    return max(
        [price[n]] + [r(i) + r (n - i) for i in range(1, n)]
    )

# max([8] + [r(1) + r(2), r(2) + r(1)])
# max([8] + [max([1]) + max([5] + [r(1) + r(1)]), max([5] + [r(1) + r(1)]) + max([1])])    # max([1]) -> 1
# max([8] + [1 + max([5] + [max([1]) + max([1])]), max([5] + [max([1]) + max([1])]) + 1])
# max([8] + [1 + max([5] + [1 + 1]), max([5] + [1 + 1]) + 1])
# max([8] + [1 + 5, 5 + 1])

In [4]:
max([8],[6,6])

[8]

In [5]:
already_computed = defaultdict(int)
solution = {}

def get_call_time(f):
    @wraps(f)
    def wrap(n):
        result = f(n)
        already_computed[(f.__name__, n)] += 1
        return result
    return wrap

In [6]:
@get_call_time
def r(n):                               #  max([(8, 0)] + [(6, 9)]) -> (8, 0) 按第一个数字排大小的
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r (n - i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (n - max_split, max_split)
    return max_price

In [9]:
r(16)

48

In [10]:
def parse_solution(n):
    left, right = solution[n]
    if right == 0: 
        return [left]
    return parse_solution(left) + parse_solution(right)

In [11]:
parse_solution(16)

[11, 3, 2]

In [25]:
# 存储已计算过的值
def memo(f):
    @wraps(f)
    def wrap(n):
        if n in m_already_computed:
            result = m_already_computed[n]
        else:
            result = f(n)
            m_already_computed[n] = result
        return result
    return wrap
        

In [26]:
solution = {}
m_already_computed = defaultdict(int)

@memo 
def r(n):                               #  max([(8, 0)] + [(6, 9)]) -> (8, 0) 按第一个数字排大小的
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r (n - i), i) for i in range(1, n)], key=lambda x: x[0]
    )
    solution[n] = (n - max_split, max_split)
    return max_price

In [27]:
r(16)

48

In [29]:
m_already_computed

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 10,
             5: 13,
             6: 17,
             7: 18,
             8: 22,
             9: 25,
             10: 30,
             11: 35,
             12: 36,
             13: 40,
             14: 43,
             15: 45,
             16: 48})

In [30]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2)}

In [31]:
parse_solution(15)

[11, 2, 2]